In [1]:
import numpy as np
import pandas as pd
import scipy.optimize 

In [1]:
class GMM_estimator():
    
    def __init__(self,data,initial_point = [0,0,0,0],t_min = 50):
        self.data = data
        self.initial_point = initial_point
        self.params = {"alpha" : self.initial_point[0],"lambda": self.initial_point[1],
                       "c":self.initial_point[2],"sigma":self.initial_point[3]}
        self.t_min = t_min
        #self.S = S

    def fit(self):
            
        def estimator(x):
    
            a,l,c,s = x
            scenario = self.data.copy()
            #S=self.S
            scenario1=scenario.copy()
    #scenario1=scenario.copy()
    
            P = 1-np.exp(-l*scenario1["FC_max"])
            scenario1["g1"] = (1+scenario1["r"])/scenario1["r"]*(scenario1["S"]/8*((1-c)/(1+s*(scenario1["n"].shift()+scenario1["n_en"])))**2-1/l)-scenario1["E"]
            scenario1["g2"] = (1+scenario1["r"])/scenario1["r"]*scenario1["S"]/8*((1-c)/(1+s*(scenario1["n"])))**2-(1/scenario1["r"]-a/(1+scenario1["r"]-a))*1/l-(1+scenario1["r"])/(1+scenario1["r"]-a)*scenario1["FC_max"]
            scenario1["g3"] = scenario1["n"]-scenario1["n"].shift()+(1-a)*(1-P)*scenario1["n"].shift()-P*scenario1["n_en"]
            scenario1["g4"] = -scenario1["FC_max"]*np.exp(-l*scenario1["FC_max"])-np.exp(-l*scenario1["FC_max"])/l+1/l-P*scenario1["FC_mean"]
    
            return scenario1["g1"].mean()**2+scenario1["g2"].mean()**2+scenario1["g3"].mean()**2+scenario1["g4"].mean()**2
        #return scenario["g1"].mean()**2,scenario["g2"].mean()**2,scenario["g3"].mean()**2,scenario["g4"].mean()**2
        
        #x1 = scipy.optimize.minimize(estimator,
                                     #x0=self.initial_point,
                                     #options = {"maxiter" : 100000},method = 'Nelder-Mead')
        
        x1 = scipy.optimize.shgo(estimator,bounds=((0,1),(0,1),(0,1),(0,1)), 
                         n=100, sampling_method='sobol', iters=10)
        
        
        
        self.params["alpha"] = x1.x[0]
        self.params["lambda"] = x1.x[1]
        self.params["c"] = x1.x[2]
        self.params["sigma"] = x1.x[3]
        
        return x1
        
    def summary(self):
        
        
        A = self.params["alpha"] 
        L = self.params["lambda"]
        S = self.params["sigma"] 
        C = self.params["c"]
        P = (1-np.exp(-L*self.data["FC_max"])).mean()
        
        print("SUMMARY", end='\n')
        print("="*39)
        print("Estimator parameters:")
        print("")
        #print(f"S: {round(self.S,7)}")
        #print(f"Bounds: {self.bounds}")
        #print(f"Initial point: {self.initial_point}")
        print(f"Zero time: {self.t_min}")
        print(f"Number of obs: {len(self.data)-self.t_min}")
        print("="*39)
        print("Data description:")
        print("")
        print(f"{(self.data[self.t_min:]).mean()}")
        print("="*39)
        print("Estimation:")
        print("")
        print(f"Alpha: {round(A,7)}")
        print(f"Lambda: {round(L,7)}")
        print(f"Sigma: {round(S,7)}")
        print(f"Marginal_cost: {round(C,7)}")
        print("="*39)
        #print("Conclusions:")
        #print("")        
        #print(f"Unconditional FC_mean: {round(1/L,7)}")
        #print(f"Probability to have a good FC: {round(P,7)}")

In [2]:
def storyteller(alpha,rlist,sigma,c,n,Slist,epoch,lambd,Elist,CF):
    
    #scale - mean FC
    #r_initial=r
    
    FC_mean = lambd
    
    FC = np.random.exponential(lambd, size=n)
    
    FC_mean_story = [FC.mean()]
    n_story = [n]
    FC_max_story = [FC.max()]
    n_ex = [0]
    n_en = [0]
    
    for i in range(epoch):
        r=rlist.iloc[i]
        E=Elist.iloc[i]
        S=Slist.iloc[i]
        #r = rlist[i]
        #if i==restep:
            #r = r2
        #if i>restep and oneshock==1:
            #r=r_initial
        
        print(f"эпоха номер: {i}")
        
        en = 0
        
        while NPV_enter(r,sigma,c,len(FC)*CF,FC_mean,S)>E:
            
            FC = np.append(FC,np.random.exponential(lambd))
            en+=1
            
        n_en.append(en*CF)
            
        print(f"вход закончился, число фирм: {len(FC)*CF}, E: {E}, NPV: {NPV_enter(r,sigma,c,len(FC)*CF,FC_mean,S)}")
        
        for i in range(len(FC)):
            
            if np.random.uniform(0,1)>alpha:
                
                FC[i] =  np.random.exponential(lambd)
                
        FC = np.flip(np.sort(FC))
        
        print(f"пересчет закончился")
        

        i=0
        k=0
        
        while i < len(FC):
            
            if NPV_exit(r,sigma,c,len(FC)*CF,FC_mean,S,FC[i])<0:
                
                FC = np.delete(FC, i)
                k+=1
            else:   
                i+=1
        
        n_ex.append(k*CF)
                
        print(f"выход закончился, число фирм: {len(FC)*CF},NPV_exit {NPV_exit(r,sigma,c,len(FC)*CF,FC_mean,S,FC.max())}")
                
        
        FC_mean_story.append(FC.mean())
        FC_max_story.append(FC.max())
        n_story.append(len(FC)*CF)
    
    stats=pd.DataFrame(data=np.array([FC_mean_story,FC_max_story,n_story,n_ex,n_en]).T, 
                        columns=["FC_mean","FC_max","n","n_ex","n_en"])    
    stats["r"] = rlist
    stats["E"] = Elist
    stats["S"] = Slist
    return stats


In [ ]:
def NPV_enter(r,sigma,c,n,FC_mean,S):
    
    return (1+r)/r*(S/8*((1+sigma*n*c)/(1+sigma*n)-c)**2-FC_mean)

In [ ]:
def NPV_exit(r,sigma,c,n,FC_mean,S,FC_i):
    
    return (1+r)/r*(S/8*((1+sigma*n*c)/(1+sigma*n)-c)**2)-(1+r)/(1+r-alpha)*FC_i-(1/r-alpha/(1+r-alpha))*FC_mean